# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [266]:
import pandas as pd
import requests
import numpy as np


# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

# Não há duplicados

(27028, 69)


(27028, 69)

In [267]:
# 2) seu código aqui

sinasc.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [268]:
# 3) seu código aqui
df = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

In [269]:
# 4) seu código aqui
df_copy = df.copy()
df_copy['APGAR5'].isna().sum()
df_copy.dropna(subset=['APGAR5'], inplace=True)
df_copy['APGAR5'].isna().sum()
df_copy['APGAR5'].shape


(26925,)

In [270]:
# 5) seu código aqui

df_copy['CONSULTAS'].fillna(9)
df_copy['ESTCIVMAE'].fillna(9)


0        5.0
1        2.0
2        9.0
3        5.0
4        2.0
        ... 
27023    2.0
27024    1.0
27025    2.0
27026    2.0
27027    5.0
Name: ESTCIVMAE, Length: 26925, dtype: float64

In [271]:
# 6) Seu código aqui
df_copy['QTDFILVIVO'].fillna(0)


0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 26925, dtype: float64

In [272]:
# 7) seu código aqui
df_copy['GRAVIDEZ'].isna().sum()
df_copy['GRAVIDEZ'].dropna()

df_copy['GESTACAO'].isna().sum()
df_copy['GESTACAO'].dropna()

# Decidi descartar os valores nulos de GRAVIDEZ e GESTAÇÂO, pois não devem impactar tanto na quantidade final de dados, apesar de serem dados importantes.


0        37 a 41 semanas
1        37 a 41 semanas
2        37 a 41 semanas
3        37 a 41 semanas
4        37 a 41 semanas
              ...       
27023    32 a 36 semanas
27024    37 a 41 semanas
27025    37 a 41 semanas
27026    32 a 36 semanas
27027    37 a 41 semanas
Name: GESTACAO, Length: 25709, dtype: object

In [273]:
# 8) seu código aqui

df_1 = df_copy.copy()

df_1['APGAR5'] = pd.to_numeric(df['APGAR5'], errors='coerce')

df_1.loc[df_1['APGAR5'] < 3, 'classificação'] = 'Asfixia Grave'
df_1.loc[(df_1['APGAR5'] >= 4) & (df_1['APGAR5'] <= 5), 'classificação'] = 'Asfixia Moderada'
df_1.loc[(df_1['APGAR5'] >= 6) & (df_1['APGAR5'] <= 7), 'classificação'] = 'Asfixia Leve'
df_1.loc[(df_1['APGAR5'] >= 8) & (df_1['APGAR5'] <= 10), 'classificação'] = 'Faixa Normal'


dummies = pd.get_dummies(df_1['APGAR5'])
pd.concat([df_1, dummies], axis=1)

df_1['classificação'].value_counts()


classificação
Faixa Normal        26463
Asfixia Leve          320
Asfixia Moderada       68
Asfixia Grave          59
Name: count, dtype: int64

In [274]:
# 9) seu código aqui
df_1.columns
df_1.rename(columns={
    'LOCNASC': 'local_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'estado_civil_mae',
    'ESCMAE': 'escolaridade_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas_realizadas'
}, inplace=True)

df_1



,local_nasc,idade_mae,estado_civil_mae,escolaridade_mae,qtd_fil_vivo,gestacao,gravidez,consultas_realizadas,APGAR5,classificação
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Faixa Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Faixa Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Faixa Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Faixa Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Faixa Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Faixa Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Faixa Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Faixa Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Faixa Normal
